In [2]:
%load_ext autoreload
%autoreload 2

import gfkmc
import numpy as np
import pandas as pd
from scipy.stats import rankdata
import random
from tree.gentree import read_tree
import math
import timeit
from typing import List
import itertools
from record_linkage import record_linkage
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
dataset = 'adult'
results_dir = f'results/{dataset}/gfkmc'
os.makedirs(results_dir, exist_ok=True)

if dataset == 'adult':
    df_orig = pd.read_csv('./datasets/adult_orig.csv').iloc[:1000]
    df = pd.read_csv('./das/adult_num_anon.csv')[df_orig.columns].iloc[:1000]
    categoricals = ['sex', 'race', 'workclass', 'marital-status', 'occupation', 'native-country', 'education']
    numericals = ['age']
    sensitives = ['salary-class']

elif dataset == 'adult_cat':
    df_orig = pd.read_csv('./datasets/adult_orig.csv').drop('age', axis=1)
    df = pd.read_csv('./das/adult_num_anon.csv')[df_orig.columns]
    categoricals = ['sex', 'race', 'workclass', 'marital-status', 'occupation', 'native-country', 'education']
    numericals = []
    sensitives = ['salary-class']

# sensitives = ['salary-class', 'race', 'sex']

k = 60
df.shape

(1000, 9)

In [4]:
df

,sex,age,race,marital-status,education,native-country,workclass,occupation,salary-class
0,Male,40,White,Never-married,Bachelors,United-States,State-gov,Adm-clerical,<=50K
1,Male,51,White,Married-civ-spouse,Bachelors,United-States,Self-emp-not-inc,Exec-managerial,<=50K
2,Male,40,White,Divorced,HS-grad,United-States,Private,Handlers-cleaners,<=50K
3,Male,51,Black,Married-civ-spouse,11th,United-States,Private,Handlers-cleaners,<=50K
4,Female,26,Black,Married-civ-spouse,Bachelors,Cuba,Private,Prof-specialty,<=50K
...,...,...,...,...,...,...,...,...,...
995,Female,45,White,Separated,Bachelors,United-States,Private,Prof-specialty,<=50K
996,Female,20,White,Divorced,HS-grad,United-States,Local-gov,Exec-managerial,<=50K
997,Male,63,White,Married-civ-spouse,HS-grad,United-States,Self-emp-not-inc,Farming-fishing,<=50K
998,Male,40,White,Married-civ-spouse,Some-college,United-States,Local-gov,Protective-serv,>50K


In [5]:
df_orig

,sex,age,race,marital-status,education,native-country,workclass,occupation,salary-class
0,Male,39,White,Never-married,Bachelors,United-States,State-gov,Adm-clerical,<=50K
1,Male,50,White,Married-civ-spouse,Bachelors,United-States,Self-emp-not-inc,Exec-managerial,<=50K
2,Male,38,White,Divorced,HS-grad,United-States,Private,Handlers-cleaners,<=50K
3,Male,53,Black,Married-civ-spouse,11th,United-States,Private,Handlers-cleaners,<=50K
4,Female,28,Black,Married-civ-spouse,Bachelors,Cuba,Private,Prof-specialty,<=50K
...,...,...,...,...,...,...,...,...,...
995,Female,43,White,Separated,Bachelors,United-States,Private,Prof-specialty,<=50K
996,Female,19,White,Divorced,HS-grad,United-States,Local-gov,Exec-managerial,<=50K
997,Male,58,White,Married-civ-spouse,HS-grad,United-States,Self-emp-not-inc,Farming-fishing,<=50K
998,Male,41,White,Married-civ-spouse,Some-college,United-States,Local-gov,Protective-serv,>50K


In [6]:
att_names = df[categoricals + sensitives].columns
att_tree = read_tree('./tree/adult/', att_names)

In [7]:
table = gfkmc.GFKMCTable(df, df_orig, numericals, categoricals, sensitives, att_tree)

In [8]:
local_start = timeit.default_timer()
remaining_groups = table.initial_clustering_phase(k)
stop = timeit.default_timer()
execution_time = stop - local_start
print(f"initial_clustering_phase execution time: {execution_time}")


initial_clustering_phase execution time: 0.010560076001638663


In [9]:

local_start = timeit.default_timer()
beta = int(len(remaining_groups) * 0.05)
table.weighting_phase(beta, remaining_groups)
stop = timeit.default_timer()
execution_time = stop - local_start
print(f"weighting_phase execution time: {execution_time}")


weighting_phase execution time: 0.25783934900027816


In [10]:
local_start = timeit.default_timer()
table.grouping_phase(k)
stop = timeit.default_timer()
execution_time = stop - local_start
print(f"grouping_phase execution time: {execution_time}")


centroid group=[1, 63, 109, 159, 187, 370, 516, 568, 579, 879, 966, 968, 979]
centroid group=[581]
centroid group=[892]
centroid group=[630]
centroid group=[931]
centroid group=[469]
centroid group=[444]
centroid group=[734]
centroid group=[417, 945]
centroid group=[99, 596]
centroid group=[601]
centroid group=[402]
centroid group=[218, 449, 462, 828]
centroid group=[141]
centroid group=[193]
centroid group=[220]
grouping_phase execution time: 14.676541773000281


In [11]:
local_start = timeit.default_timer()
table.adjustment_phase()
stop = timeit.default_timer()
execution_time = stop - local_start
print(f"adjustment_phase execution time: {execution_time}")


adjustment_phase execution time: 1.8215617930000008


In [12]:
# gen_method = 'most_common_values'
gen_method = 'cluster_centroid'

df_anon = table.cluster_generalization(gen_method)

# df_anon['cluster'] = -1
# for i, cluster in enumerate(table.clusters):
#     df_anon.loc[cluster.r_indices, 'cluster'] = i
#     print(f"cluster={i}, len={cluster.size}")

ncp_value = table.ncp(df_anon)
print(ncp_value)

# df_anon.to_csv(f'./{results_dir}/adult_anon_k{k}_test.csv', index=False)



# with open(f'./{results_dir}/adult_anon_k{k}.txt', 'w') as f:
#     f.write(f'{ncp_value}\n{matches}')


0.23010404783781196


In [13]:
matches = record_linkage(df_orig, df_anon, att_tree, numericals, categoricals)
print(matches)

dn execution time: 0.0008788880004431121s
dc execution time: 0.02982614000029571s
0 of 1000
dn execution time: 0.0006173319998197258s
dc execution time: 0.02839319700069609s
1 of 1000
dn execution time: 0.0005669059992214898s
dc execution time: 0.02966117299911275s
2 of 1000
dn execution time: 0.0005943529995420249s
dc execution time: 0.028901575000418234s
3 of 1000
dn execution time: 0.0005609690015262458s
dc execution time: 0.02848880900091899s
4 of 1000
dn execution time: 0.0005967279994365526s
dc execution time: 0.02864721100013412s
5 of 1000
dn execution time: 0.0006689450001431396s
dc execution time: 0.029192256000897032s
6 of 1000
dn execution time: 0.0006075530000089202s
dc execution time: 0.028215169000759488s
7 of 1000
dn execution time: 0.0005504929995368002s
dc execution time: 0.02826629399896774s
8 of 1000
dn execution time: 0.0005534960000659339s
dc execution time: 0.028683040000032634s
9 of 1000
dn execution time: 0.0006424050006899051s
dc execution time: 0.0285599090002

In [14]:
# for i, cluster in enumerate(table.clusters):
#     intra_diss = table.intra_diss(cluster)
#     print(f'table.intra_diss(table.clusters[{i}]) = {intra_diss}')

In [15]:
# for pair in itertools.combinations(range(len(table.clusters)), 2):
#     inter_diss = table.inter_diss(table.clusters[pair[0]], table.clusters[pair[1]])
#     print(f'table.inter_diss(table.clusters[{pair[0]}], table.clusters[{pair[1]}]) = {inter_diss}')